In [2]:
print("OK!")

OK!


In [3]:
# langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.0.225 which is incompatible.
# so don't use langchain-community until installed a compatible version
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
from langchain_pinecone import Pinecone 
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\jerry\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [19]:
from dotenv import load_dotenv, find_dotenv 
load_dotenv()
#load_dotenv(find_dotenv()) # if above doesn't work. use this one. 

True

In [20]:
import os 

PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV=os.getenv("PINECONE_API_ENV")

In [21]:
PINECONE_API_KEY


'6676aad1-d50f-4832-adc5-fd132a56b982'

In [6]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:

os.getcwd()

'd:\\AI\\Llama\\project_medical_chatbot\\End-to-end-Medical-Chatbot-using-Llama2\\research'

In [8]:
extracted_data = load_pdf("../small_data")

In [29]:
#check data 
extracted_data[10:12]

[Document(metadata={'source': '..\\small_data\\Attention is all you need.pdf', 'page': 10}, page_content='[5]Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk,\nand Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical\nmachine translation. CoRR , abs/1406.1078, 2014.\n[6]Francois Chollet. Xception: Deep learning with depthwise separable convolutions. arXiv\npreprint arXiv:1610.02357 , 2016.\n[7]Junyoung Chung, Çaglar Gülçehre, Kyunghyun Cho, and Yoshua Bengio. Empirical evaluation\nof gated recurrent neural networks on sequence modeling. CoRR , abs/1412.3555, 2014.\n[8]Chris Dyer, Adhiguna Kuncoro, Miguel Ballesteros, and Noah A. Smith. Recurrent neural\nnetwork grammars. In Proc. of NAACL , 2016.\n[9]Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin. Convolu-\ntional sequence to sequence learning. arXiv preprint arXiv:1705.03122v2 , 2017.\n[10] Alex Graves. Generating sequences with rec

In [9]:
# Create text chunks 
def text_split(extracted_data) : 
  return  RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20).split_documents(extracted_data)


In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 243


In [32]:
# text_chunks
text_chunks[100:105]

[Document(metadata={'source': '..\\small_data\\MachineTranslationwithAttention.pdf', 'page': 1}, page_content='a ﬁxed-length vector and a decoder then outputs\na translation from the encoded vector, where in\nmost of the cases the encoder and decoder are\nmainly implemented as RNNs, CNNs or self-\nattention network (Wu et al., 2018). The whole\nencoder–decoder system, which consists of the\nencoder and the decoder for a language pair, is'),
 Document(metadata={'source': '..\\small_data\\MachineTranslationwithAttention.pdf', 'page': 2}, page_content='jointly trained to maximize the probability of a\ncorrect translation given a source sentence (Bah-\ndanau et al., 2014). After the initial proposal by\n(Sutskever et al., 2014; Bahdanau et al., 2014),\nmuch work has been done on the sequence-to-\nsequence neural machine translation model rang-\ning from new attention mechanism (Luong et al.,\n2015) to working on the problem of out-of-\nvocabulary words (Jean et al., 2015) for which se-\nqu

In [11]:
#download embedding model

model_name="sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)


c:\Users\jerry\anaconda3\envs\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [72]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [36]:
print(query_result)

[-0.03447727486491203, 0.03102317824959755, 0.006734970025718212, 0.026108985766768456, -0.03936202451586723, -0.16030244529247284, 0.06692401319742203, -0.006441489793360233, -0.0474504791200161, 0.014758856035768986, 0.07087527960538864, 0.05552763119339943, 0.019193334504961967, -0.026251312345266342, -0.01010954286903143, -0.02694045566022396, 0.022307461127638817, -0.022226648405194283, -0.14969263970851898, -0.017493007704615593, 0.00767625542357564, 0.05435224249958992, 0.0032543970737606287, 0.031725890934467316, -0.0846213847398758, -0.02940601296722889, 0.05159561336040497, 0.04812406003475189, -0.0033148222137242556, -0.058279167860746384, 0.04196927323937416, 0.022210685536265373, 0.1281888335943222, -0.022338971495628357, -0.011656315997242928, 0.06292839348316193, -0.032876335084438324, -0.09122604131698608, -0.031175347045063972, 0.0526994913816452, 0.04703482985496521, -0.08420311659574509, -0.030056199058890343, -0.02074483036994934, 0.009517835453152657, -0.0037217906

In [13]:
#query_result
#Initializing the Pinecone

pc=pinecone.Pinecone(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV,
              region="us-east-1")


In [15]:
index_name="medical-chatbot"

#index = pc.Index(index_name)

In [20]:
#!pip install langchain_pinecone 

In [16]:

#Creating Embeddings for Each of The Text Chunks & storing

# already loaded. use next line
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'fe04a9aa159813bba603c6a86616d693', 'Date': 'Sun, 25 Aug 2024 02:10:26 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [40]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"
query = "Why sometime I can feel ichy"


docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='In autonomic dysreflexia, irritation of the skin,\nbowel, or bladder causes a highly exaggerated responsefrom autonomic nerves. This response is caused by theuncontrolled release of norepinephrine, a hormone simi-lar to adrenaline. Uncontrolled release of norepinephrinecauses a rapid rise in blood pressure and a slowing of theheart rate. These symptoms are accompanied by throb-bing headache , nausea, anxiety , sweating, and goose'), Document(page_content='head (fontanel) may be depressed. Symptoms of dehy-\ndration at any age include cracked lips, dry or sticky\nmouth, lethargy, and sunken eyes. A person who is\ndehydrated cries without shedding tears and does not\nurinate very often. The skin is less elastic than it should\nbe and is slow to return to its normal position after\nbeing pinched.\nDehydration can cause confusion, constipation , dis-\ncomfort, drowsiness, fever, and thirst. The skin turns pale'), Document(page_content='bing pain caused by irr

In [41]:
for d in docs: print(d.page_content); print("\n -- EOL -- \n")

In autonomic dysreflexia, irritation of the skin,
bowel, or bladder causes a highly exaggerated responsefrom autonomic nerves. This response is caused by theuncontrolled release of norepinephrine, a hormone simi-lar to adrenaline. Uncontrolled release of norepinephrinecauses a rapid rise in blood pressure and a slowing of theheart rate. These symptoms are accompanied by throb-bing headache , nausea, anxiety , sweating, and goose

 -- EOL -- 

head (fontanel) may be depressed. Symptoms of dehy-
dration at any age include cracked lips, dry or sticky
mouth, lethargy, and sunken eyes. A person who is
dehydrated cries without shedding tears and does not
urinate very often. The skin is less elastic than it should
be and is slow to return to its normal position after
being pinched.
Dehydration can cause confusion, constipation , dis-
comfort, drowsiness, fever, and thirst. The skin turns pale

 -- EOL -- 

bing pain caused by irritation of or damage to a nerve.
The pain is usually brief but m

In [76]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [77]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [78]:
# which one? 
from langchain.llms import CTransformers
# don't use langchain-community.  it's not compatible. 
#from langchain_community.llms import CTransformers
#from transformers import AutoModelForCausalLM


In [95]:
%pip install ctransformers==0.2.24
#!pip install ctransformers==0.2.5

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/10.6 MB 8.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/10.6 MB 5.6 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/10.6 MB 6.8 MB/s eta 0:00:01
   --------------------- ------------------ 5.8/10.6 MB 6.9 MB/s eta 0:00:01
   -------------------------- ------------- 7.1/10.6 MB 6.8 MB/s eta 0:00:01
   --------------------------------- ------ 8.9/10.6 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: ctransformers
    Found existing installation: ctransformers 0.2.5
    Uninstalling ctransformers-0.2.5:
      Successfully uninstalled ctransformers-0.2.5


  You can safely remove it manually.


In [104]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama", gpu_layers=0)


print(llm("AI is going to"))

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


 help us and our businesses in so many ways. surely there will be a future where all the mundane tasks are taken care of by AI and we can focus on more creative and fulfilling work.
Reply	6 months ago
I couldn't agree more! The potential benefits of AI are vast, and it's exciting to think about how it could shape the future of work and society as a whole. From automating repetitive tasks to providing personalized recommendations and improving decision-making processes, AI has the potential to revolutionize the way we live and work.
Of course, there are also challenges and concerns associated with the adoption of AI, such as job displacement and ethical considerations. However, I believe that with careful planning, implementation, and regulation, these challenges can be addressed and the benefits of AI can be realized while minimizing its negative impacts.
Overall, I'm optimistic about the future of work and the role that AI will play in shaping it. With the right strategies and approac

In [127]:
print(llm("I got a mosquito bite. I feel very itchy. why"))

In [126]:

#llm2 = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q4_0.bin")
#llm3 = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q4_0.bin", config={'max_new_tokens':512, "temperature":0.1})
llm4 = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q4_K_S.bin", config={'max_new_tokens':512, "temperature":0.1})
# GGUF doesn't work. 
#llm4 = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGUF", model_file="llama-2-7b-chat.Q4_K_M.gguf", config={'max_new_tokens':512, "temperature":0.1})


Fetching 1 files: 100%|██████████| 1/1 [04:48<00:00, 288.79s/it]


RuntimeError: Failed to create LLM 'llama' from 'C:\Users\jerry\.cache\huggingface\hub\models--TheBloke--Llama-2-7B-Chat-GGML\snapshots\76cd63c351ae389e1d4b91cab2cf470aab11864b\llama-2-7b-chat.ggmlv3.q4_K_S.bin'.

In [ ]:
# note: llm from AutoModelForCausalLM is not usable. use llm2 or llm3 

In [120]:
qa=RetrievalQA.from_chain_type(
    llm=llm3, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [121]:
#dont execute this loop !! just run 1 iteration 

#while True:
    
user_input=input(f"Input Prompt:")
result=qa.invoke({"query": user_input})
print("Response : ", result["result"])

Response :  The itchiness associated with a mosquito bite is likely due to the body's immune response to the saliva injected into the skin during the feeding process. The saliva contains an anti-coagulant that prevents blood from clotting, which can cause irritation and inflammation at the bite site. Additionally, some people may be more sensitive to mosquito bites than others, which could contribute to a greater itchiness.
